In [ ]:
import pandas as pd
import itertools
import numpy as np
from itertools import chain
from collections import defaultdict
from numpy import unique
import pandahouse as ph
from datetime import date
import holidays
from fbprophet import Prophet


# 1-ОЕ ЗАДАНИЕ

In [ ]:
market = pd.read_csv('https://stepik.org/media/attachments/lesson/409319/test1_completed.csv')

In [ ]:
market.rename(columns = {'Товар':'product_name', 'Количество':'count_of_product'}, inplace = True)

## План работы:
1) Сначала перебрать всевозможные пары продуктов для одного пользователя (id)  
2) Применить это действие для всех пользователя (id)  
3) Объединить эти пары в один датафрейм  
4) Создать функцию, которая считает сколько раз встречается конкретная пара продуктов  
5) Найти 5 наиболе встречающихся пар  


In [ ]:
'''Функция, которая превращает все продукты каждого пользователя в список
Входные данные: id клиента
Выходные данные: список покупок в виде list
'''
def func_id_little(id):
    list1 = market[market.id == id].product_name.tolist()
    return list1

In [ ]:
'''Функция, которая превращает все продукты каждого пользователя в список списков, внутри которых кортежи с уникальными парами
Входные данные: id клиента
Выходные данные: кортежи уникальных пар
'''
list_finish_1 = [list(itertools.combinations(x, 2)) for x in [func_id_little(x) for x in market.id]]

In [ ]:
'''Функция, которая делит список с кортежом tuple на списки с первыми элементами пар каждого id
Входные данные: список с кортежом tuple 
Выходные данные: кортеж со списком первого элемента и со списком второго элемента пары
'''
def table_maker(list_tuple):
    list_1 = [x[0] for x in list_tuple]
    list_2 = [x[1] for x in list_tuple]
    return list_1, list_2

In [ ]:
'''Функция образует результирующую таблицу со всеми уникальными парами всех покупателей
Входные данные:  
Выходные данные: таблица со всеи уникальными парами
'''
def final_table_maker():
    list_3 = []
    list_4 = []
    for my_tuple in list_finish_1:
        list_3.extend(table_maker(my_tuple)[0])
        list_4.extend(table_maker(my_tuple)[1])
    table_df = pd.DataFrame({'product_1': list_3, 'product_2': list_4})
    return table_df

In [ ]:
%%time
united_table = final_table_maker()

In [ ]:
united_table.head()

In [ ]:
dictionary = defaultdict(int) #специальная структура словаря
def itertuples_func(df):
    for row in df.itertuples(name='ride', index = False):
        myList = unique(list(row)) #преобразует каждую строку в список и оставляет только ункальные значения списка, чтобы не было разницы в [1, a] и [a, 1]
        myString = ','.join(map(str, myList)) #переводит словарь в строку, где покупки отделяются запятой 
        dictionary[myString] += 1 #считает кличество покупок
    return dictionary

In [ ]:
%%time
itertuples_func(united_table)

In [ ]:
'''Функция преобразует получившийся словарь в датафрейм (финальную таблицу)
Входные данные: словарь
Выходные данные: финальная таблица
'''

def result_table(dictionary):
    list_product_1 = [] #создаем пустые словари
    list_product_2 = []
    list_count = []
    for key in dictionary:
            dictionary_List = key.split(',') #превращаем обратно строку в список
            list_product_1.append(dictionary_List[0]) # создаем список из первых покупок
            list_product_2.append(dictionary_List[1]) # список из вторых покупок
            list_count.append(dictionary.get(key)) # список из количества повторов
    df4_Final = pd.DataFrame({'product_1': list_product_1, 
                              'product_2': list_product_2, 
                              'count': list_count})
    return df4_Final

In [ ]:
final_table = result_table(dictionary)

In [ ]:
final_table.sort_values('count', ascending=False).head()

# 2-ОЕ ЗАДАНИЕ

In [ ]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/426638/test2_completed.csv')

In [ ]:
df.head()

In [ ]:
df.plot()

In [ ]:
df = df.rename(columns= {'Дата': 'ds', 'Число заказов': 'y'})

In [ ]:
skidki = pd.DataFrame({'holiday': 'skidki', 'ds': pd.to_datetime(['2020-02-02', '2020-02-09']),
                       'lower_window' : 0, 'upper_window' : 7 })
holidays_dict = holidays.RU(years=(2020, 2021))
df_holidays = pd.DataFrame.from_dict(holidays_dict, orient='index') \
    .reset_index()
df_holidays = df_holidays.rename({'index':'ds', 0:'holiday'}, axis ='columns')
df_holidays['ds'] = pd.to_datetime(df_holidays.ds)
df_holidays = df_holidays.sort_values(by=['ds'])
df_holidays = df_holidays.reset_index(drop=True)
holidays = pd.concat([df_holidays, skidki])

In [ ]:
m = Prophet()
m.fit(df)

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
m = Prophet(holidays=holidays, weekly_seasonality=True, n_changepoints = 10)
forecast = m.fit(df).predict(future)

In [ ]:
fig1 = m.plot(forecast)
fig2 = m.plot_components(forecast)

In [ ]:
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df.set_index('ds'))
cmp_df['e'] = cmp_df['y'] - cmp_df['yhat']
cmp_df['p'] = 100*cmp_df['e']/cmp_df['y']
print('MAPE (средняя абсолютная ошибка в процентах) – ', np.mean(abs(cmp_df[-90:]['p'])),'%')
print('MAE (средняя абсолютная ошибка) – ', np.mean(abs(cmp_df[-90:]['e'])))

#### Таким образом, мы смогли предсказать нашу модель сос редней ошибкой в ~4%

#### Изобразим нашу модель с помощью plotly

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected = True)

iplot([
    go.Scatter(x=df['ds'], y=df['y'], name='fact'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='prediction'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=forecast['ds'], y=forecast['trend'], name='trend')
])

# 3-ЬЕ ЗАДАНИЕ

In [1]:
import pandas as pd
import numpy as np
import pandahouse as ph
from datetime import date

In [2]:
# Объявляем параметры подключения к clickhouse
connection = {'database':'project_variant_2',
                  'host':'https://clickhouse.lab.karpov.courses',
                  'user':'student',
                  'password':'dpo_python_2020'}

In [3]:
# Вкидываем данные (первый столбец - даты (ds), второй - уники (y))
query = """
SELECT     a.purchase_id AS purchase_id,
           a.client_id AS client_id,
           a.purchase_date AS purchase_date, 
           CAST(a.quantity AS Int16) AS quantity,
           CAST(a.revenue AS Int16) AS revenue,
           a.client_category AS client_category,
           CAST(b.client_age AS Int16) AS client_age,
           CAST(b.client_registration AS Int16) client_registration_age, /* в днях */
           c.promotion_name AS promotion_name,
           c.category_name AS category_name,
           c.partner_name AS partner_name,
           d.client_city AS client_city,
           d.city AS city
    FROM  (            
                SELECT l.purchase_id AS purchase_id,
                       l.client_id AS client_id,
                       l.city_id AS city_id,
                       l.purchase_date AS purchase_date, 
                       l.category_id AS category_id,
                       l.quantity AS quantity,
                       l.revenue AS revenue,
                       r.client_category AS client_category
                       
                FROM   (    
                
                        SELECT purchase_id,
                               client_id,
                               city_id,
                               promotion_id,
                               purchase_date, 
                               category_id,
                               quantity,
                               price*quantity AS revenue
                        FROM project_variant_2.purchase
                        WHERE    (purchase_date >= '01.05.2020'
                                   OR
                                  purchase_date <= '01.08.2020')
                              AND
                                 (status == 1)
                        ) AS l
                JOIN    (
                        SELECT client_id,
                               CASE WHEN count(purchase_id) == 1
                                    THEN 'new'
                                    ELSE  'old' 
                               END as client_category
                        FROM project_variant_2.purchase
                        WHERE (purchase_date >= '01.05.2020'
                                   OR
                               purchase_date <= '01.08.2020')
                            AND
                              (status == 1)
                        GROUP BY client_id
                        ) AS r
                ON l.client_id = r.client_id
                ) AS a
JOIN    (
        SELECT *,
           CASE WHEN formatDateTime(now(), '%m-%d') < formatDateTime(birth_date, '%m-%d')
                THEN (YEAR(now()) - YEAR(birth_date)) - 1
                ELSE  YEAR(now()) - YEAR(birth_date)
           END as client_age,
           
           DateDiff(Day,registration,now()) as client_registration /* будем считать, что клиент начал пользоваться магазином, когда зарегестрировался */
           
        FROM project_variant_2.client
        ) AS b
    ON a.client_id = b.client_id
JOIN project_variant_2.promotion AS c
    ON a.category_id = c.category_id
JOIN project_variant_2.city AS d
    ON a.city_id = d.city_id 
LIMIT 10
"""
df = ph.read_clickhouse(query=query, connection=connection)
df

,purchase_id,client_id,purchase_date,quantity,revenue,client_category,client_age,client_registration_age,promotion_name,category_name,partner_name,client_city,city
0,0,853,2020-06-05,5,150,old,29,828,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Щучинщина,Киев
1,0,853,2020-06-05,5,150,old,29,828,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Москва,Киев
2,0,853,2020-06-05,5,150,old,29,828,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Ярославль,Киев
3,0,853,2020-06-05,5,150,old,29,828,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Санкт-Петербург,Киев
4,0,853,2020-06-05,5,150,old,29,828,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Новгород,Киев
5,0,853,2020-06-05,5,150,old,29,828,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Минск,Киев
6,0,853,2020-06-05,5,150,old,29,828,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Киев,Киев
7,0,853,2020-06-05,5,150,old,29,828,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Казань,Киев
8,0,853,2020-06-05,5,150,old,29,828,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Вена,Киев
9,1,406,2020-06-15,5,1150,old,24,703,campaign_4,Прочее,KarpovCourses,Минск,Новгород
